<a href="https://colab.research.google.com/github/nabiilazen/sanberlearn/blob/main/Sanbercode_FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df_train = pd.read_csv('/content/drive/My Drive/PEKAN 4/pendapatan_train.csv')
df_train.head()


,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,40,Wiraswasta,205047,SMA,9,Menikah LDR,Perbaikan Kerajinan,Laki2,0.0,0.0,40.0,>7jt
1,1,33,Pemerintah Lokal,298785,10th,6,Cerai,Supir,Laki2,0.0,0.0,40.0,<=7jt
2,2,26,Wiraswasta,106856,D4,11,Belum Pernah Menikah,Pemuka Agama,Perempuan,0.0,0.0,40.0,<=7jt
3,3,54,Pekerja Bebas Perusahaan,117674,Sarjana,13,Menikah,Sales,Laki2,0.0,0.0,40.0,>7jt
4,4,48,Wiraswasta,268234,SMA,9,Cerai,Perbaikan Kerajinan,Laki2,0.0,0.0,50.0,<=7jt


In [ ]:
print(df_train.shape)

(28794, 13)


In [ ]:
#mengecek missing value
df_train.isna().sum()

id                       0
Umur                     0
Kelas Pekerja            0
Berat Akhir              0
Pendidikan               0
Jmlh Tahun Pendidikan    0
Status Perkawinan        0
Pekerjaan                0
Jenis Kelamin            0
Keuntungan Kapital       0
Kerugian Capital         0
Jam per Minggu           0
Gaji                     0
dtype: int64

In [ ]:
print(df_train['Kelas Pekerja'].unique())
print(df_train['Pendidikan'].unique())
print(df_train['Status Perkawinan'].unique())
print(df_train['Pekerjaan'].unique())
print(df_train['Jenis Kelamin'].unique())

['Wiraswasta' 'Pemerintah Lokal' 'Pekerja Bebas Perusahaan'
 'Pekerja Bebas Bukan Perusahan' 'Pemerintah Negara' '?'
 'Pemerintah Provinsi' 'Tanpa di Bayar' 'Tidak Pernah Bekerja']
['SMA' '10th' 'D4' 'Sarjana' 'Pendidikan Tinggi' 'D3' '1st-4th' '5th-6th'
 '9th' '11th' 'Sekolah Professional' '12th' 'Master' '7th-8th' 'Doktor'
 'SD']
['Menikah LDR' 'Cerai' 'Belum Pernah Menikah' 'Menikah' 'Berpisah' 'Janda']
['Perbaikan Kerajinan' 'Supir' 'Pemuka Agama' 'Sales' 'Pembersih'
 'Servis Lainnya' 'Tech-support' 'Spesialis' 'Ekesekutif Managerial' '?'
 'Petani' 'Mesin Inspeksi' 'Penjaga' 'Asisten Rumah Tangga' 'Tentara']
['Laki2' 'Perempuan']


In [ ]:
#encoding categorical data
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

non_numerikal_data_features = ['Kelas Pekerja','Pendidikan','Status Perkawinan','Pekerjaan','Jenis Kelamin']

for data in non_numerikal_data_features:        
    df_train[data] = LabelEncoder().fit_transform(df_train[data])

In [ ]:
print(df_train['Kelas Pekerja'].unique())
print(df_train['Pendidikan'].unique())
print(df_train['Status Perkawinan'].unique())
print(df_train['Pekerjaan'].unique())
print(df_train['Jenis Kelamin'].unique())

[8 3 2 1 4 0 5 6 7]
[13  0  8 14 11  7  3  4  6  1 15  2 10  5  9 12]
[5 2 0 4 1 3]
[ 7 12  5  9  4 10 13 11  2  0  8  3  6  1 14]
[0 1]


In [ ]:
df_train.head()

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,40,8,205047,13,9,5,7,0,0.0,0.0,40.0,>7jt
1,1,33,3,298785,0,6,2,12,0,0.0,0.0,40.0,<=7jt
2,2,26,8,106856,8,11,0,5,1,0.0,0.0,40.0,<=7jt
3,3,54,2,117674,14,13,4,9,0,0.0,0.0,40.0,>7jt
4,4,48,8,268234,13,9,2,7,0,0.0,0.0,50.0,<=7jt


In [ ]:
#split data X
X = StandardScaler().fit_transform(df_train.drop(columns=['Gaji']))

In [ ]:
X.shape

(28794, 12)

In [ ]:
#mengubah nilai gaji
gaji_dict = {'<=7jt':'0',
             '>7jt':'1'}

df_train['Gaji'] = df_train['Gaji'].replace(gaji_dict)

In [ ]:
df_train['Gaji'].value_counts()

0    21973
1     6821
Name: Gaji, dtype: int64

In [ ]:
#split data y
y = df_train.iloc[:, -1].values

In [ ]:
y

array(['1', '0', '0', ..., '0', '0', '0'], dtype=object)

**Train test split**

In [ ]:
#Membagi dataset menjadi data test dan data training
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=21, stratify=y)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def bestparam_randCV(model,hyperparam,x_train, y_train, n_iter=1000):
    
    hyperparam = hyperparam
    randomizedCV = RandomizedSearchCV(model, param_distributions = hyperparam, cv = 10,
                                             n_iter = n_iter, scoring = 'roc_auc', n_jobs=-1, 
                                             random_state = 42, verbose = True)
    randomizedCV.fit(x_train, y_train)
    
    #print (randomizedCV.cv_results_)
    print ('Best Accuracy', randomizedCV.score(x_train, y_train))
    print ('Best Param', randomizedCV.best_params_)
    return randomizedCV

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model         = RandomForestClassifier()             

hyperparam = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
} 
n_iter     = 10
best_rfc  = bestparam_randCV(model, hyperparam, X_train, y_train, n_iter)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 15.0min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 38.0min finished


Best Accuracy 0.9863322871206912
Best Param {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': True}


In [ ]:
rfc = RandomForestClassifier(bootstrap    = best_rfc.best_params_.get('bootstrap'),
                             max_depth    = best_rfc.best_params_.get('max_depth'),
                             max_features = best_rfc.best_params_.get('max_features'),
                             min_samples_leaf  = best_rfc.best_params_.get('min_samples_leaf'),
                             min_samples_split = best_rfc.best_params_.get('min_samples_split'),
                             n_estimators     = best_rfc.best_params_.get('n_estimators'))

In [ ]:
rfc

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
result_rfc = rfc.fit(X_train, y_train)

In [ ]:
y_predict_train = rfc.predict(X_train)
y_predict_test = rfc.predict(X_test)

In [ ]:
print('Score data train', accuracy_score(y_train, y_predict_train))
print('Score data test', accuracy_score(y_test, y_predict_test))

Score data train 0.9361448772016869
Score data test 0.8591272137978933


In [ ]:
df_test = pd.read_csv('/content/drive/My Drive/PEKAN 4/pendapatan_test.csv')
df_test.head()

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu
0,28794,31,Pemerintah Provinsi,207301,SMA,9,Menikah,Pemuka Agama,Perempuan,0.0,0.0,40.0
1,28795,34,Wiraswasta,324254,SMA,9,Menikah,Mesin Inspeksi,Laki2,0.0,0.0,40.0
2,28796,58,Wiraswasta,27385,SMA,9,Menikah,Ekesekutif Managerial,Laki2,0.0,0.0,60.0
3,28797,49,Wiraswasta,723746,SMA,9,Menikah,Perbaikan Kerajinan,Laki2,0.0,0.0,43.0
4,28798,60,Wiraswasta,194980,SMA,9,Berpisah,Pemuka Agama,Perempuan,0.0,0.0,20.0


In [ ]:
print(df_test.shape)

(9599, 12)


In [ ]:
df_test.isna().sum()

id                       0
Umur                     0
Kelas Pekerja            0
Berat Akhir              0
Pendidikan               0
Jmlh Tahun Pendidikan    0
Status Perkawinan        0
Pekerjaan                0
Jenis Kelamin            0
Keuntungan Kapital       0
Kerugian Capital         0
Jam per Minggu           0
dtype: int64

In [ ]:
print(df_test['Kelas Pekerja'].unique())
print(df_test['Pendidikan'].unique())
print(df_test['Status Perkawinan'].unique())
print(df_test['Pekerjaan'].unique())
print(df_test['Jenis Kelamin'].unique())

['Pemerintah Provinsi' 'Wiraswasta' 'Pekerja Bebas Bukan Perusahan'
 'Pemerintah Lokal' 'Pekerja Bebas Perusahaan' 'Pemerintah Negara' '?'
 'Tanpa di Bayar' 'Tidak Pernah Bekerja']
['SMA' 'Pendidikan Tinggi' '11th' 'Sarjana' '10th' 'Master' '7th-8th'
 '9th' '12th' 'D3' 'Doktor' 'D4' 'Sekolah Professional' '1st-4th'
 '5th-6th' 'SD']
['Menikah' 'Berpisah' 'Cerai' 'Belum Pernah Menikah' 'Menikah LDR' 'Janda']
['Pemuka Agama' 'Mesin Inspeksi' 'Ekesekutif Managerial'
 'Perbaikan Kerajinan' 'Spesialis' 'Sales' 'Penjaga' 'Supir' 'Pembersih'
 'Petani' 'Servis Lainnya' 'Tech-support' '?' 'Asisten Rumah Tangga'
 'Tentara']
['Perempuan' 'Laki2']


In [ ]:
#encoding categorical data
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

non_numerikal_data_features = ['Kelas Pekerja','Pendidikan','Status Perkawinan','Pekerjaan','Jenis Kelamin']

for data in non_numerikal_data_features:        
    df_test[data] = LabelEncoder().fit_transform(df_test[data])

In [ ]:
print(df_test['Kelas Pekerja'].unique())
print(df_test['Pendidikan'].unique())
print(df_test['Status Perkawinan'].unique())
print(df_test['Pekerjaan'].unique())
print(df_test['Jenis Kelamin'].unique())

[5 8 1 3 2 4 0 6 7]
[13 11  1 14  0 10  5  6  2  7  9  8 15  3  4 12]
[4 1 2 0 5 3]
[ 5  3  2  7 11  9  6 12  4  8 10 13  0  1 14]
[1 0]


In [ ]:
df_test.head()

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu
0,28794,31,5,207301,13,9,4,5,1,0.0,0.0,40.0
1,28795,34,8,324254,13,9,4,3,0,0.0,0.0,40.0
2,28796,58,8,27385,13,9,4,2,0,0.0,0.0,60.0
3,28797,49,8,723746,13,9,4,7,0,0.0,0.0,43.0
4,28798,60,8,194980,13,9,1,5,1,0.0,0.0,20.0


In [ ]:
X_test = StandardScaler().fit_transform(df_test)

In [ ]:
df_test_predict = rfc.predict(X_test)

In [ ]:
df_test['Gaji'] = df_test_predict

In [ ]:
df_test['Gaji']

0       0
1       0
2       0
3       0
4       0
       ..
9594    1
9595    1
9596    0
9597    0
9598    0
Name: Gaji, Length: 9599, dtype: object

In [ ]:
result =pd.DataFrame({
    'Id':df_test['id'],
    'Gaji':df_test['Gaji']
})

In [ ]:
result.tail()

,Id,Gaji
9594,38388,1
9595,38389,1
9596,38390,0
9597,38391,0
9598,38392,0


In [ ]:
result.to_csv("submission_nabila.csv", index=False)